#**KopyKat**: Peptide Conformation Generation 
[OpenMM](http://docs.openmm.org/latest/userguide/)🔗Integrated.  GPU⚡Accelerated  <img src="https://t3.ftcdn.net/jpg/04/83/66/74/240_F_483667401_w8c04vlYURjZT7bZ9es9Fp0sL7gxkl8s.jpg" height="190" align="right" style="height:100px"> 

*By: Mithony Keng*
<img src="https://t3.ftcdn.net/jpg/04/83/66/74/240_F_483667401_w8c04vlYURjZT7bZ9es9Fp0sL7gxkl8s.jpg" height="235" align="right" style="height:100px"><img src="https://t3.ftcdn.net/jpg/04/83/66/74/240_F_483667401_w8c04vlYURjZT7bZ9es9Fp0sL7gxkl8s.jpg" height="285" align="right" style="height:100px">

In [ ]:
#@title Install Conda for Colaab and other requirements
#Install Conda for Colab and other requirements.
#Run cell required for conda program

try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass
!pip install py3Dmol
!pip install nglview
!pip install parmed

In [ ]:
#@title Import the installations
try:
    import condacolab
    from google.colab import files
    from IPython.display import clear_output
    condacolab.check()
    !conda install -q -y -c conda-forge mdtraj openmm openmmforcefields pdbfixer pypdb rdkit
    on_colab = True
    clear_output()  # clear the excessive installation outputs
    print("Dependencies successfully installed!")
except ModuleNotFoundError:
    on_colab = False

import sys
import os.path
try:
  dir1 = 'results'
  os.mkdir(dir1)
except FileExistsError:
  next
try:
  dir2 = 'data'
  os.mkdir(dir2)
except FileExistsError:
  next
  

# **Generation** 

**KopyKat takes in a pdb formated file. After the file name has been entered, run the cell to preprocess the input structure.**

In [ ]:
#@title
import pdbfixer
import simtk.openmm as mm
import simtk.openmm.app as app
from simtk.openmm import unit
from openff.toolkit.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator
import numpy.linalg as lin
import sys
import os
import os.path
import math
from pkg_resources import resource_filename
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile

Enter_File = "6a5j.pdb" #@param {type:"string"}

file_name =('{}'.format(Enter_File))
file_name = file_name.strip()
new_file = file_name.replace('.pdb','_conf.pdb')
#fixer = PDBFixer(pdbid='2hyy')
fixer = PDBFixer(filename=('{}'.format(file_name)))
fixer.findMissingResidues()
#fixer.findNonstandardResidues()
#fixer.replaceNonstandardResidues()
#fixer.removeHeterogens()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
#fixer.addSolvent(fixer.topology.getUnitCellDimensions()) 
#fixer.addSolvent(padding=2*unit.angstrom, ionicStrength=0.050*unit.molar)
PDBFile.writeFile(fixer.topology, fixer.positions, open('data/{}'.format(file_name), 'w'))

**This platform uses the [OpenMM](http://docs.openmm.org/latest/userguide/application/02_running_sims.html) Molecular Dynamics software for conformation generation. Complete the fields below to initiate conformation generation.** 

In [ ]:
#@title
#@markdown Select a temperature for the MD trajectory
Temperature_Kelvin = 454 #@param {type:"slider", min:200, max:700, step:1}

#@markdown Choose a primary force field (for gas phase, select only the primary Force_Field_1) 

Force_Field_1 ="amber14/protein.ff14SB.xml" #@param ["","amber14/protein.ff14SB.xml","amber14-all.xml", "amber96.xml", "charmm36.xml", "amoeba2018.xml"] {allow-input: true}
f1 = Force_Field_1
""
#@markdown For solution phase explicit solvent model, add the following force field
Force_Field_2 = "" #@param ["", "amber14/tip3p.xml", "amber14/tip3pfb.xml", "amber14/tip4pew.xml", "amber14/tip4pfb.xml", "amber14/spce.xml", "charmm36/tip3p-pme-b.xml", "charmm36/tip3p-pme-f.xml","charmm36/tip4pew.xml", "charmm36/tip4p2005.xml", "charmm36/tip5p.xml", "charmm36/tip5pew.xml"] {allow-input: true}
f2 = Force_Field_2

#@markdown For solution phase implicit solvent model, add the following force field
Force_Field_3 ="" #@param ["","implicit/hct.xml", "implicit/obc1.xml", "implicit/obc2.xml", "implicit/gbn.xml","implicit/gbn2.xml"] {allow-input: true}
f3 = Force_Field_3

#@markdown Structure minimization
Energy_Minimize = True #@param {type:"boolean"}

#@markdown Enter the number of conformation to generate
Number_of_Conformers =  1000#@param {type:"integer"}


from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import py3Dmol
import nglview
import nglview as nv

num_conf = Number_of_Conformers

while True:
  try:
    num_conf = int(num_conf)
    break
  except ValueError:
    print("Please try again.")
    num_conf = input('Enter the conformation to generate: ')
pdb = PDBFile('data/{}'.format(file_name))
modeller = app.Modeller(pdb.topology, pdb.positions)
if f1 !="" and f2 =="" and f3 =="":
  forcefield = app.ForceField(f1)
if f1 !="" and f2 !="" and f3 =="":
  forcefield = app.ForceField(f1,f2)
if f1 !="" and f2 =="" and f3 !="":
  forcefield = app.ForceField(f1,f3)
if f1 =="" and f2 !="" and f3 =="":
  forcefield = app.ForceField("amber14-all.xml",f2)
if f1 =="" and f2 =="" and f3 !="":
  forcefield = app.ForceField("amber14-all.xml",f3)
  print("Since no primary force field was selected, amber14-all.xml has be implemented as default.")
try:
  system = forcefield.createSystem(modeller.topology)
except ValueError:
  next 
integrator = mm.LangevinIntegrator(Temperature_Kelvin * kelvin, 1.0 / picoseconds, 2.0 * femtoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

if Energy_Minimize == True:
  simulation.minimizeEnergy()
else:
  next
  
steps = num_conf * 100  # corresponds to 100 ps
simulation.reporters.append(PDBReporter('results/{}'.format(new_file), 100))
print("")
print("")
print("Generated conformers will be appended to a single pdb file.")
simulation.step(steps)
print("")
print("Task complete.")
files.download("results/{}".format(new_file))
# simulation.reporters.append(StateDataReporter(stdout, 20, step=True,
#         potentialEnergy=True, temperature=True))

#view = nv.show_file('results/{}'.format(new_file))  # load "3pqr" from RCSB PDB and display viewer widget
#view